In [442]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [ ]:
# separate columns by ;
df = pd.read_csv('../atis_dataset.csv', sep=';')

In [444]:
df.head()

,atis_id,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
0,1,KIAD,10.0,6,0,180,>6000,19,L,CAT III,5.9,1.0,-1.0,NaN,1,0,0,1
1,1,KIAD,10.0,6,0,180,>6000,19,C,CAT III,5.9,1.0,-1.0,NaN,1,0,0,1
2,1,KIAD,10.0,6,0,180,>6000,19,R,CAT III,5.9,1.0,-1.0,NaN,1,0,0,1
3,2,KLAX,10.0,4,0,40,>6000,6,R,No ILS,3.8,1.4,-1.0,NaN,0,1,1,0
4,3,KSFO,10.0,8,0,260,>6000,28,L,CAT III,7.5,2.7,-1.0,NaN,1,0,0,0


In [445]:
df.iloc[22]

atis_id                         15
airport_icao                  KATL
visibility                    10.0
wind_speed                       3
wind_gust                        0
wind_direction                 240
rvr                         >6000 
runway_designator_number        28
runway_designator_side         NaN
runway_ils_category         No ILS
headwind                       2.3
crosswind                      1.9
ceiling                       -1.0
weather_phenomenon             NaN
ILS                              0
RNAV                             0
RNP                              0
VISUAL                           1
Name: 22, dtype: object

In [446]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6613 entries, 0 to 6612
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   atis_id                   6613 non-null   int64  
 1   airport_icao              6613 non-null   object 
 2   visibility                6613 non-null   float64
 3   wind_speed                6613 non-null   int64  
 4   wind_gust                 6613 non-null   int64  
 5   wind_direction            6613 non-null   int64  
 6   rvr                       6613 non-null   object 
 7   runway_designator_number  6613 non-null   int64  
 8   runway_designator_side    6041 non-null   object 
 9   runway_ils_category       6613 non-null   object 
 10  headwind                  6613 non-null   float64
 11  crosswind                 6613 non-null   float64
 12  ceiling                   6609 non-null   float64
 13  weather_phenomenon        583 non-null    object 
 14  ILS     

In [447]:
# check null values before preprocess
df.isnull().sum()

atis_id                        0
airport_icao                   0
visibility                     0
wind_speed                     0
wind_gust                      0
wind_direction                 0
rvr                            0
runway_designator_number       0
runway_designator_side       572
runway_ils_category            0
headwind                       0
crosswind                      0
ceiling                        4
weather_phenomenon          6030
ILS                            0
RNAV                           0
RNP                            0
VISUAL                         0
dtype: int64

In [448]:
# delete rows where rvr = FFF
df = df[df['rvr'] != 'FFF']

In [ ]:
# change N/A weather_phenomenon to "No Phenomenon"
df['weather_phenomenon'] = df['weather_phenomenon'].fillna('No Phenomenon')

In [450]:
# fill empty runway_designator_side to X
df['runway_designator_side'] = df['runway_designator_side'].fillna('X') # X means no L, R, or C

In [451]:
# clear whitespaces in rvr column
df['rvr'] = df['rvr'].str.strip()

In [452]:
# extract non-digit characters only (excluding decimal point)
df['rvr_tendency'] = df['rvr'].astype(str).apply(lambda x: ''.join(re.findall(r'\D+', x)).strip())
df['rvr'] = df['rvr'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)

In [453]:
# change rvr_tendency empty string ('') to 'Stable'
df.loc[df.rvr_tendency == '', 'rvr_tendency'] = 'Stable'

# change rvr_tendency '>' symbol to 'Excellent'
df.loc[df.rvr_tendency == '>', 'rvr_tendency'] = 'Excellent'

# change rvr_tendency '▲' symbol to 'Improving'
df.loc[df.rvr_tendency == '▲', 'rvr_tendency'] = 'Improving'

# change rvr_tendency '▼' symbol to 'Deteriorating'
df.loc[df.rvr_tendency == '▼', 'rvr_tendency'] = 'Deteriorating'

In [454]:
# remove the column temporarily
rvr_tendency = df.pop('rvr_tendency')

# find the index of 'rvr' column
rvr_index = df.columns.get_loc('rvr')

# insert it right after 'rvr'
df.insert(rvr_index + 1, 'rvr_tendency', rvr_tendency)


In [455]:
# drop atis_id column
df.drop(columns=['atis_id'], inplace=True)

In [456]:
# update KIAD runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KIAD)
# Step 1: Set all ILS runways at KIAD to CAT I by default
kiad_condition = (df['airport_icao'] == 'KIAD')
df.loc[kiad_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Explicit ILS capability mapping
ils_mapping_kiad = {
    ('KIAD', 1, 'C'): 'CAT II',
    ('KIAD', 1, 'L'): 'CAT III',
    ('KIAD', 1, 'R'): 'CAT III',
    ('KIAD', 19, 'C'): 'CAT III',
    ('KIAD', 19, 'L'): 'CAT II',
    ('KIAD', 19, 'R'): 'CAT III',
}

# Step 3: Apply mapping
def update_ils_category_kiad(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kiad.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kiad, axis=1)

df.loc[
    (df['airport_icao'] == 'KIAD') & (df['runway_designator_number'] == 19) & (df['runway_designator_side'] == 'L'), 'runway_ils_category'
] = 'CAT II'

In [ ]:
# update KLAX runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KLAX)
# Set all KLAX runways with ILS/LOC to CAT I by default
df.loc[(df['airport_icao'] == 'KLAX'), 'runway_ils_category'] = 'CAT I'

# Then upgrade those with CAT III capability
df.loc[(df['airport_icao'] == 'KLAX') & (df['runway_designator_number'].isin([24, 25])) &
    (df['runway_designator_side'] == 'R') | (df['runway_designator_side'] == 'L'),
    'runway_ils_category'] = 'CAT III'

In [458]:
# update KATL runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KATL)
# Step 1: Default all ILS runways at KATL to CAT I
katl_condition = (df['airport_icao'] == 'KATL')
df.loc[katl_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Override based on max available CAT
ils_mapping_katl = {
    ('KATL', 8, 'L'): 'CAT III',
    ('KATL', 9, 'R'): 'CAT III',
    ('KATL', 10, 'X'): 'CAT III',
    ('KATL', 26, 'R'): 'CAT II',
    ('KATL', 27, 'L'): 'CAT II',
    ('KATL', 28, 'X'): 'CAT II',
}

def update_ils_category_katl(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_katl.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_katl, axis=1)

In [459]:
# update KSFO runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KSFO)
# Step 1: Default all KSFO ILS runways to CAT I
ksfo_condition = (df['airport_icao'] == 'KSFO')
df.loc[ksfo_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Manual override for better-equipped runways
ils_mapping_ksfo = {
    ('KSFO', 28, 'L'): 'CAT II',
    ('KSFO', 28, 'R'): 'CAT III',
}

# Step 3: Apply the mapping
def update_ils_category_ksfo(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_ksfo.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_ksfo, axis=1)

In [460]:
# update KORD runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KORD)
# Step 1: Default all KORD ILS runways to CAT I
kord_condition = (df['airport_icao'] == 'KORD')
df.loc[kord_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Override based on specific mappings
ils_mapping_kord = {
    ('KORD', 4, 'R'): 'CAT II',
    ('KORD', 9, 'C'): 'CAT III',
    ('KORD', 9, 'L'): 'CAT III',
    ('KORD', 9, 'R'): 'CAT III',
    ('KORD', 10, 'C'): 'CAT III',
    ('KORD', 10, 'L'): 'CAT III',
    ('KORD', 10, 'R'): 'CAT III',
    ('KORD', 22, 'L'): 'CAT II',
    ('KORD', 22, 'R'): 'CAT I',
    ('KORD', 27, 'C'): 'CAT III',
    ('KORD', 27, 'L'): 'CAT III',
    ('KORD', 27, 'R'): 'CAT III',
    ('KORD', 28, 'C'): 'CAT III',
    ('KORD', 28, 'L'): 'CAT III',
    ('KORD', 28, 'R'): 'CAT III',
}

def update_ils_category_kord(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kord.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kord, axis=1)

In [461]:
# update KJFK runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KJFK)
# Step 1: Default all ILS runways at KJFK to CAT I
kjfk_condition = (df['airport_icao'] == 'KJFK')
df.loc[kjfk_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Specific overrides based on actual CAT info
ils_mapping_kjfk = {
    ('KJFK', 13, 'L'): 'CAT II',
    ('KJFK', 4, 'R'): 'CAT III',
    ('KJFK', 22, 'L'): 'CAT III',
}

# Step 3: Apply mapping
def update_ils_category_kjfk(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kjfk.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kjfk, axis=1)

In [463]:
# check after preprocessed
df.head()

,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,rvr_tendency,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
0,KIAD,10.0,6,0,180,6000.0,Excellent,19,L,CAT III,5.9,1.0,-1.0,None,1,0,0,1
1,KIAD,10.0,6,0,180,6000.0,Excellent,19,C,CAT III,5.9,1.0,-1.0,None,1,0,0,1
2,KIAD,10.0,6,0,180,6000.0,Excellent,19,R,CAT III,5.9,1.0,-1.0,None,1,0,0,1
3,KLAX,10.0,4,0,40,6000.0,Excellent,6,R,CAT I,3.8,1.4,-1.0,None,0,1,1,0
4,KSFO,10.0,8,0,260,6000.0,Excellent,28,L,CAT II,7.5,2.7,-1.0,None,1,0,0,0


In [464]:
df.iloc[22]

airport_icao                     KATL
visibility                       10.0
wind_speed                          3
wind_gust                           0
wind_direction                    240
rvr                            6000.0
rvr_tendency                Excellent
runway_designator_number           28
runway_designator_side              X
runway_ils_category            CAT II
headwind                          2.3
crosswind                         1.9
ceiling                          -1.0
weather_phenomenon               None
ILS                                 0
RNAV                                0
RNP                                 0
VISUAL                              1
Name: 22, dtype: object

In [465]:
# check null values after preprocessed
df.isnull().sum()

airport_icao                0
visibility                  0
wind_speed                  0
wind_gust                   0
wind_direction              0
rvr                         0
rvr_tendency                0
runway_designator_number    0
runway_designator_side      0
runway_ils_category         0
headwind                    0
crosswind                   0
ceiling                     4
weather_phenomenon          0
ILS                         0
RNAV                        0
RNP                         0
VISUAL                      0
dtype: int64

In [466]:
# see rows with null value in any column
df[df.isnull().any(axis=1)]

,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,rvr_tendency,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
4390,KJFK,1.0,6,0,160,6000.0,Stable,22,L,CAT III,3.0,5.2,NaN,BR,1,0,0,0
4391,KJFK,1.0,6,0,160,6000.0,Excellent,22,R,CAT I,3.0,5.2,NaN,BR,1,0,0,0
5897,KJFK,1.0,12,0,130,3000.0,Improving,22,L,CAT III,0.0,12.0,NaN,- DZ BR,1,0,0,0
5898,KJFK,1.0,12,0,130,2000.0,Deteriorating,22,R,CAT I,0.0,12.0,NaN,- DZ BR,1,0,0,0


In [468]:
# delete rows that have null value(s)
df.dropna()

,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,rvr_tendency,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
0,KIAD,10.0,6,0,180,6000.0,Excellent,19,L,CAT III,5.9,1.0,-1.0,None,1,0,0,1
1,KIAD,10.0,6,0,180,6000.0,Excellent,19,C,CAT III,5.9,1.0,-1.0,None,1,0,0,1
2,KIAD,10.0,6,0,180,6000.0,Excellent,19,R,CAT III,5.9,1.0,-1.0,None,1,0,0,1
3,KLAX,10.0,4,0,40,6000.0,Excellent,6,R,CAT I,3.8,1.4,-1.0,None,0,1,1,0
4,KSFO,10.0,8,0,260,6000.0,Excellent,28,L,CAT II,7.5,2.7,-1.0,None,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,KORD,10.0,12,0,150,6000.0,Excellent,9,L,CAT III,6.0,10.4,8000.0,None,1,0,0,1
6609,KORD,10.0,12,0,150,6000.0,Excellent,10,C,CAT III,7.7,9.2,8000.0,None,1,0,0,1
6610,KJFK,10.0,15,23,330,6000.0,Excellent,31,R,CAT I,14.1,5.1,-1.0,None,1,0,0,1
6611,KJFK,10.0,15,23,330,6000.0,Excellent,31,L,CAT I,14.1,5.1,-1.0,None,1,0,0,1


In [469]:
# see columns datatypes after preprocessed
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6591 entries, 0 to 6612
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   airport_icao              6591 non-null   object 
 1   visibility                6591 non-null   float64
 2   wind_speed                6591 non-null   int64  
 3   wind_gust                 6591 non-null   int64  
 4   wind_direction            6591 non-null   int64  
 5   rvr                       6591 non-null   float64
 6   rvr_tendency              6591 non-null   object 
 7   runway_designator_number  6591 non-null   int64  
 8   runway_designator_side    6591 non-null   object 
 9   runway_ils_category       6591 non-null   object 
 10  headwind                  6591 non-null   float64
 11  crosswind                 6591 non-null   float64
 12  ceiling                   6587 non-null   float64
 13  weather_phenomenon        6591 non-null   object 
 14  ILS          

In [470]:
df.describe()

,visibility,wind_speed,wind_gust,wind_direction,rvr,runway_designator_number,headwind,crosswind,ceiling,ILS,RNAV,RNP,VISUAL
count,6591.000000,6591.000000,6591.000000,6591.000000,6591.000000,6591.000000,6591.000000,6591.000000,6587.000000,6591.000000,6591.000000,6591.000000,6591.000000
mean,9.476900,9.667880,7.619785,206.415870,5959.748141,20.107116,6.505492,5.239266,5597.774860,0.742528,0.109695,0.077075,0.656350
std,1.819284,5.626002,12.048984,100.944406,367.597124,9.669064,5.903299,4.505573,8682.266004,0.437275,0.312533,0.266730,0.474962
min,0.250000,0.000000,0.000000,-1.000000,0.000000,1.000000,-10.400000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,5.000000,0.000000,150.000000,6000.000000,10.000000,2.500000,1.900000,-1.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,9.000000,0.000000,220.000000,6000.000000,24.000000,6.000000,4.000000,-1.000000,1.000000,0.000000,0.000000,1.000000
75%,10.000000,13.000000,20.000000,290.000000,6000.000000,28.000000,10.000000,7.700000,7500.000000,1.000000,0.000000,0.000000,1.000000
max,10.000000,29.000000,43.000000,360.000000,6000.000000,31.000000,29.000000,25.600000,30000.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Save the cleaned DataFrame
df.to_csv('../../datasets/atis_dataset_preprocessed.csv', sep=';', index=False)
df.to_csv('atis_dataset_preprocessed.csv', sep=';', index=False)